<a href="https://colab.research.google.com/github/MrKkondae/practiceLLM/blob/main/miniLLM_GPT2_JSON_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 라이브러리 설치 및 환경 설정

In [ ]:
!pip install transformers datasets accelerate --upgrade

import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1


2. KoGPT2 모델 및 토크나이저 로드

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>'
)

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

3. 파인튜닝용 대화 데이터셋 가져오기
📌 1단계: Colab에서 파일 업로드

In [ ]:
from google.colab import files

# 파일 업로드 창 열기
uploaded = files.upload()

📌 2단계: JSON 파일을 Hugging Face Dataset으로 로딩

In [ ]:
import json
from datasets import Dataset

# 파일 읽기
with open("대화_데이터셋_UTF8.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Dataset으로 변환
dataset = Dataset.from_list(data)

# 확인
print(dataset[0])


4. 데이터셋 토큰화

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# ✨ text 컬럼 제거 + tokenizer 출력 유지
tokenized_dataset = dataset.map(tokenize_function, remove_columns=["text"])

5. 학습설정 및 Trainer 구성

In [ ]:
training_args = TrainingArguments(
    output_dir="./kogpt2-finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    logging_steps=5,
    save_steps=10,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)


6. 파인튜닝 실행

In [ ]:
trainer.train()

✅ 7. 파인튜닝된 모델로 응답 테스트

In [ ]:
def chat_kogpt2(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(model.device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response[len(prompt):].strip()



# 대화 루프 시작
print("🗨️ KoGPT2 챗봇입니다. '종료'라고 입력하면 끝나요.")
while True:
    user_input = input("👤 사용자: ")
    if user_input.strip() == "종료":
        print("🤖 챗봇: 안녕히 가세요!")
        break
    full_prompt = f"사용자: {user_input}\n챗봇:"
    response = chat_kogpt2(full_prompt)
    print(f"🤖 챗봇: {response}"